In [1]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf


In [2]:
# Import data
data_dir = 'E:/AI/00/data/Mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting E:/AI/00/data/Mnist/input_data\train-images-idx3-ubyte.gz
Extracting E:/AI/00/data/Mnist/input_data\train-labels-idx1-ubyte.gz
Extracting E:/AI/00/data/Mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting E:/AI/00/data/Mnist/input_data\t10k-labels-idx1-ubyte.gz


In [4]:
# Define loss and optimizer
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
learning_rate = tf.placeholder(tf.float32)
# 把输入转为 4维张量
with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

In [5]:
# First convolutional layer - maps one grayscale image to 32 feature maps.
# output [-1, 28, 28, 32]
with tf.name_scope('conv1'):
    # 卷积核为[5,5], 将1通道转为32通道
    shape = [5, 5, 1, 32]
    W_conv1 = tf.Variable(tf.truncated_normal(shape, stddev=0.1),
                          collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    shape = [32]
    b_conv1 = tf.Variable(tf.constant(0.1, shape=shape))
    l_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], # 步长为 1
                           padding='SAME') + b_conv1
    h_conv1 = tf.nn.relu(l_conv1)

In [6]:
# Pooling layer - downsamples by 2X.
# output [-1, 14, 14, 32]
with tf.name_scope('pool1'):
    # 最大池化, 核为[2,2], 步长为2 
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='VALID')

In [8]:
# Second convolutional layer -- maps 32 feature maps to 64.
# output [-1, 14, 14, 64]
with tf.name_scope('conv2'):
    W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1),
                          collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    l_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], 
                           padding='SAME') + b_conv2
    h_conv2 = tf.nn.relu(l_conv2)

In [9]:
# Second pooling layer.
# output [-1, 7, 7, 64]
with tf.name_scope('pool2'):
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')

In [10]:
# Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
# is down to 7x7x64 feature maps -- maps this to 1024 features.
# output [1024]
with tf.name_scope('fc1'):
    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [11]:
# Dropout - controls the complexity of the model, prevents co-adaptation of
# features.
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [12]:
# Map the 1024 features to 10 classes, one for each digit
# output [10]
with tf.name_scope('fc2'):
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1),
                        collections=[tf.GraphKeys.GLOBAL_VARIABLES,'WEIGHTS'])
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

    y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [15]:
# The raw formulation of cross-entropy,
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection('WEIGHTS')] )
total_loss = cross_entropy + 7e-5*l2_loss

# 优化方法,随机梯度下降
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
#train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
#train_step = tf.train.MomentumOptimizer(learning_rate).minimize(total_loss)

# 全部变量初始化
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [14]:
# Train
for step in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    lr = 0.01
    _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
    if (step+1) % 100 == 0:
        print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
              (step+1, loss, l2_loss_value, total_loss_value))
      # Test trained model
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(accuracy, feed_dict={x: batch_xs, 
                                            y_: batch_ys, 
                                            keep_prob:0.5}))
    if (step+1) % 1000 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                            y_: mnist.test.labels, 
                                            keep_prob:0.5}))

step 100, entropy loss: 0.854040, l2_loss: 12656.458008, total loss: 1.739992
0.72
step 200, entropy loss: 0.562304, l2_loss: 12654.958984, total loss: 1.448151
0.78
step 300, entropy loss: 0.402332, l2_loss: 12653.348633, total loss: 1.288067
0.92
step 400, entropy loss: 0.309610, l2_loss: 12651.752930, total loss: 1.195233
0.94
step 500, entropy loss: 0.405611, l2_loss: 12650.087891, total loss: 1.291118
0.91
step 600, entropy loss: 0.215452, l2_loss: 12648.442383, total loss: 1.100843
0.93
step 700, entropy loss: 0.319619, l2_loss: 12646.762695, total loss: 1.204893
0.88
step 800, entropy loss: 0.325464, l2_loss: 12645.062500, total loss: 1.210619
0.88
step 900, entropy loss: 0.222727, l2_loss: 12643.357422, total loss: 1.107762
0.95
step 1000, entropy loss: 0.079866, l2_loss: 12641.640625, total loss: 0.964781
0.97
0.9361
step 1100, entropy loss: 0.174286, l2_loss: 12639.932617, total loss: 1.059081
0.97
step 1200, entropy loss: 0.331232, l2_loss: 12638.271484, total loss: 1.215911